# Blood glucose level challenge - A Deep Learning Approach for Blood Glucose Prediction of Type 1 Diabetes

## Authors

* Albertetti Fabrizio
* Freiburghaus Jonas
* Rizzotti Aïcha

In [ ]:
from bs4 import BeautifulSoup
import datetime
import keras.backend as K
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import Conv1D, Dense, Input, MaxPooling1D, LSTM
from keras.models import load_model, Sequential
from keras.optimizers import Adam
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np
import pandas as pd
import os
from scipy.ndimage import gaussian_filter1d
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

## Build dataset

Read the four used features from an XML file :

* Glucose level
* Basal
* Bolus
* Meal

It is possible to add other available features by reading them in the same manner.  
All the resulting dataframes are then merged in one single dataframe.

The folder structure should be as following :

* Blgp.ipynb
* OhioT1DM-testing
    * 559-ws-testing.xml
    * 563-ws-testing.xml
    * 570-ws-testing.xml
    * 575-ws-testing.xml
    * 588-ws-testing.xml
    * 591-ws-testing.xml
* OhioT1DM-training
    * 559-ws-training.xml
    * 563-ws-training.xml
    * 570-ws-training.xml
    * 575-ws-training.xml
    * 588-ws-training.xml
    * 591-ws-training.xml
* OhioT1DM-2-testing
    * 540-ws-testing.xml
    * 544-ws-testing.xml
    * 552-ws-testing.xml
    * 567-ws-testing.xml
    * 584-ws-testing.xml
    * 596-ws-testing.xml
* OhioT1DM-2-training
    * 540-ws-training.xml
    * 544-ws-training.xml
    * 552-ws-training.xml
    * 567-ws-training.xml
    * 584-ws-training.xml
    * 596-ws-training.xml

In [ ]:
def read_cgm(soup):
    df_cgm = pd.DataFrame([event.attrs for event in soup.find('glucose_level').find_all('event')])
    df_cgm['ts'] = pd.to_datetime(df_cgm['ts'], format='%d-%m-%Y %H:%M:%S')
    df_cgm.index = df_cgm['ts']
    df_cgm.drop('ts', axis=1, inplace=True)
    df_cgm.rename(columns={'value': 'glucose_level'}, inplace=True)
    df_cgm['glucose_level'] = df_cgm['glucose_level'].apply(np.float32)
    return df_cgm

def read_basal(soup):
    df_basal = pd.DataFrame([event.attrs for event in soup.find('basal').find_all('event')])
    df_basal['ts'] = pd.to_datetime(df_basal['ts'], format='%d-%m-%Y %H:%M:%S')
    df_basal.index = df_basal['ts']
    df_basal.drop('ts', axis=1, inplace=True)
    df_basal.rename(columns={'value': 'basal'}, inplace=True)
    df_basal['basal'] = df_basal['basal'].apply(np.float32)
    return df_basal

def read_bolus(soup):
    df_bolus = pd.DataFrame([event.attrs for event in soup.find('bolus').find_all('event')])
    df_bolus.rename(columns={'ts_begin': 'ts', 'dose': 'bolus'}, inplace=True)
    df_bolus['ts'] = pd.to_datetime(df_bolus['ts'], format='%d-%m-%Y %H:%M:%S')
    df_bolus.index = df_bolus['ts']
    df_bolus.drop(['ts', 'ts_end', 'type'], axis=1, inplace=True)
    df_bolus['bolus'] = df_bolus['bolus'].apply(np.float32)
    return df_bolus

def read_meal(soup):
    try:
        df_meal = pd.DataFrame([event.attrs for event in soup.find('meal').find_all('event')])
        df_meal['ts'] = pd.to_datetime(df_meal['ts'], format='%d-%m-%Y %H:%M:%S')
        df_meal.index = df_meal['ts']
        df_meal.drop(['ts', 'type'], axis=1, inplace=True)
        df_meal['carbs'] = df_meal['carbs'].apply(np.float32)
        return df_meal
    except KeyError:
        df_meal = pd.DataFrame({'ts': [], 'carbs': []})
        df_meal.index = df_meal['ts']
        return df_meal.drop('ts', axis=1)

In [ ]:
def parse_contributor(contrib_dir, contrib_file):
    with open(os.path.join(contrib_dir, contrib_file)) as file:
        soup = BeautifulSoup(file.read(), 'lxml')

        df_cgm = read_cgm(soup)
        df_basal = read_basal(soup)
        df_bolus = read_bolus(soup)
        df_meal = read_meal(soup)

        df_contributor = df_meal.join([df_bolus, df_basal, df_cgm], how='outer')
        
        resample_index = pd.date_range(start= df_contributor.index[0], end= df_contributor.index[-1], freq='5T')
        df_dummy = pd.DataFrame(np.NaN, index=resample_index, columns= df_contributor.columns)
        df_contributor =  df_contributor.combine_first(df_dummy)
        
        return df_contributor

In [ ]:
def parse_directory(contrib_dir):
    contrib_files = {contrib_file.split('-')[0]:contrib_file for contrib_file in os.listdir(contrib_dir)}
    
    contrib_dataframes = {}
    for contrib_id, contrib_file in contrib_files.items():
        contrib_dataframes[contrib_id] = parse_contributor(contrib_dir, contrib_file)
        
    return contrib_dataframes

In [ ]:
all_df_train_stage1 = parse_directory('OhioT1DM-training')
all_df_test_stage1 = parse_directory('OhioT1DM-testing')
all_df_train_stage2 = parse_directory('OhioT1DM-2-training')
all_df_test_stage2 = parse_directory('OhioT1DM-2-testing')

## Preprocessing

The preprocessing is summarized in the following steps :

For the features :

1. Save the indexes where the glucose level is available
2. Resample the dataframe to an 1 second delta time
3. Forward fill the values
4. Fill the left NaN values with 0
5. Resample to a 5 minutes delta time
6. In order to preserve as much information as possible. The carbs, basal and bolus values are assigned to the closest past time point.
7. A custom keras TimeseriesGenerator is implemented to smooth the window of the past 2 hours worth data. We use a Gaussian smoothing with std = 1

For the targets :

Compute the difference from the last blood glucose level known.

$$y_{t+L} = bg_{t+L} - bg_{t}, \mathrm{for} \ L = 1, 2, \ \dots, 12$$

Where $bg_t$ is the blood glucose value at time $t$, $L$ is the lag value in timesteps for the horizon, and $y$ the label to predict, that is the differentiated value of the blood glucose level.

The model will predict the blood glucose level difference and the predicted blood glucose is then obtained by :

$$\widehat{bg}_{t+L} = bg_{t} + \hat{y}_{t+L}, \mathrm{for} \ L = 1, 2, \ \dots, 12$$

In [ ]:
def align_series(df):
    interest_index = df[~df['glucose_level'].isna()].index
    df_cp = df.copy(deep=True)
    
    df = df.resample('1S').pad()
    df = df.fillna(method='ffill').fillna(0)
    df.loc[df.index[-1] + pd.Timedelta('5T'), 'glucose_level'] = df.iloc[-1]['glucose_level']
    df = df.resample('5T').first().shift(-1)
    
    for feature in ['carbs', 'basal', 'bolus']:
        locs = [df.index.get_loc(idx, method='ffill') for idx in df_cp[~df_cp[feature].isna()].index[1:]]
        mask_df = df.reset_index().index.isin(locs)
        df.loc[~mask_df, feature] = 0
        
    return df, interest_index

In [ ]:
def lag_target(df, lag=12, target_col='glucose_level'):
    target_df = pd.concat([-df[target_col].diff(periods=-i) for i in range(1, lag + 1)], axis=1).dropna(axis=0)
    df = df.iloc[:len(target_df)]

    return df, target_df

In [ ]:
class SmoothedTimeseriesGenerator(TimeseriesGenerator):
    
    def __init__(self, sigma=1, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.sigma = sigma
    
    def __getitem__(self, idx):
        x, y = super().__getitem__(idx)
        for i in range(x.shape[0]):
            for j in range(x.shape[2]):
                x[i, :, j] = gaussian_filter1d(x[i, :, j], self.sigma)
        
        return x, y
    
def make_generator(X, Y, horizon_window=24, sampling_rate=1, batch_size=1024):
    generator = SmoothedTimeseriesGenerator(data=X, targets=Y.shift(1).values, length=horizon_window, sampling_rate=sampling_rate, batch_size=batch_size)
    return generator

## Baseline model

In the paper the reported persistent algorithm does not use the time aligned features.

Persistent algorithm :

$$\hat{y}_{t+L} = bg_{t-1}, \mathrm{for} \ L = 1, 2, \ \dots, 12$$

In [ ]:
def compute_persistence_metrics(all_df_test):
    pred_times = list(range(1, 13))
    mae_dict = {f'mae+{i}' : [] for i in pred_times}
    rmse_dict = {f'rmse+{i}' : [] for i in pred_times}
    
    for df_test in tqdm(all_df_test.values()):
        df_test, test_interest_index = align_series(df_test)
        
        for i in pred_times:
            df_test_cp = df_test.copy(deep=True)
            df_test_cp[f'glucose_level+{i}'] = df_test_cp['glucose_level'].shift(i)
            df_test_cp = df_test_cp.resample('1S').ffill() 
            
            df_pred = df_test_cp.loc[test_interest_index].copy(deep=True)
            df_pred.dropna(inplace=True)
            y_true = df_pred['glucose_level'].values
            y_pred = df_pred[f'glucose_level+{i}'].values
            
            rmse = np.sqrt(
                mean_squared_error(
                    y_true,
                    y_pred
                )
            )
            rmse_dict[f'rmse+{i}'].append(rmse)

            mae = mean_absolute_error(
                y_true, 
                y_pred
            )
            mae_dict[f'mae+{i}'].append(mae)
            
    df_rmse = pd.DataFrame(rmse_dict)
    df_mae = pd.DataFrame(mae_dict)
    
    return df_rmse, df_mae

In [ ]:
df_rmse, df_mae = compute_persistence_metrics(all_df_test_stage2)

In [ ]:
df_mae.mean(axis=0)

In [ ]:
df_rmse.mean(axis=0)

## Model

In [ ]:
class DeltaLSTM(LSTM):

    def __init__(self, units, **kwargs):
        super(DeltaLSTM, self).__init__(units, **kwargs)

    def step(self, x, states):
        h, [h, c] = super(DeltaLSTM, self).step(x, states)
        o = h / self.activation(c)
        h = K.multiply(o, K.tanh(c))
        y = K.update_add(K.multiply([o, K.tanh(c)]), x)

        return y, [h, c]

In [ ]:
def build_model(n_timesteps=24, n_features=4):
    model = Sequential()

    model.add(Conv1D(filters=8, kernel_size=24, padding='same', input_shape=(n_timesteps, n_features)))
    model.add(MaxPooling1D())
    model.add(Conv1D(filters=16, kernel_size=12, padding='same'))
    model.add(MaxPooling1D())
    model.add(Conv1D(filters=32, kernel_size=6, padding='same'))
    model.add(MaxPooling1D())
    model.add(DeltaLSTM(64))
    model.add(Dense(256))
    model.add(Dense(32))
    model.add(Dense(12))

    model.compile(
        optimizer='RMSProp',
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'],
    )

    return model

## First Ohio T1DM dataset

If desired a model may be pretrained using the first Ohio T1DM dataset.

In [ ]:
def pretrain_model(all_train_df, all_test_df, epochs=1000, shuffle=False):
    COLUMNS = ['carbs', 'bolus', 'basal', 'glucose_level']
    cb_lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor= 0.1, patience=3, min_lr= 1e-9)
    cb_early_stopping = EarlyStopping(
        monitor='val_loss',
        min_delta=0,
        patience=50,
        verbose=0,
        mode='auto',
        baseline=None,
        restore_best_weights=True
    )
    crnn_model = build_model()
    
    for train_contrib_id, test_contrib_id in zip(all_train_df.keys(), all_test_df.keys()):
        df_train = all_train_df[train_contrib_id]
        df_train['glucose_level'] = df_train['glucose_level'].interpolate('linear')
        df_train, train_interest_index = align_series(df_train)
        df_train.dropna(inplace=True)
        X_train, Y_train = lag_target(df_train[COLUMNS])
        train_generator = make_generator(X_train.values, Y_train)

        df_test = all_test_df[test_contrib_id]
        df_test, test_interest_index = align_series(df_test)
        df_test.dropna(inplace=True)
        X_test, Y_test = lag_target(df_test[COLUMNS])
        test_generator = make_generator(X_test.values, Y_test)
        
        print(f'Fitting contributor : {train_contrib_id}')

        crnn_model.fit_generator(train_generator,
                                 steps_per_epoch=len(train_generator),
                                 epochs=epochs,
                                 validation_data=test_generator,
                                 shuffle=shuffle,
                                 callbacks=[cb_lr_reducer, cb_early_stopping],
                                 verbose=1)
        
    return crnn_model

In [ ]:
crnn_model = pretrain_model(all_df_train_stage1, all_df_test_stage1)

In [ ]:
crnn_model.save('rcnn_pretrain.h5')

## Second Ohio T1DM dataset

The pretrained model is loaded for each contributor and trained over 1000 epochs with a batch size of 1024.  
Early stopping is used to regularize the model. The patience is set to 50 epcohs and the weights from the model with the best validation loss are restored.  
Before early stopping a learning rate reducer is used with a patience of 15 epochs.  
The train-validation split is performed chronologically wise.

In [ ]:
def run_predictions(crnn_model, times, test_generator, df_test, horizon_window=24):
    y_pred = crnn_model.predict_generator(test_generator)
    df_pred = df_test.copy()
    for time_idx, time in times.items():
        pred_col = f"pred_cgm_{time}"
        df_pred['shift'] = df_pred['glucose_level'].shift(time + 1)
        df_pred[pred_col] = df_pred['shift'][horizon_window:] + y_pred[:, time]
        df_pred = df_pred.drop(columns=['shift'])

    return df_pred

In [ ]:
def compute_metric(times, df_pred, horizon_window=24):
    rmse_by_time = []
    mae_by_time = []
    for time_idx, time in times.items():
        pred_col = f"pred_cgm_{time}"
        start_idx = horizon_window + time

        mse = mean_squared_error(df_pred[pred_col], df_pred['glucose_level'])
        rmse_by_time.append(np.sqrt(mse))

        mae = mean_absolute_error(df_pred[pred_col], df_pred['glucose_level'])
        mae_by_time.append(mae)

    return pd.DataFrame({"Times": list(times.keys()), "RMSE": rmse_by_time, "MAE": mae_by_time})

In [ ]:
def dump_df(df, path):
    df.to_csv(path, sep=',', mode='a')

In [ ]:
def evaluate(
    contributor, 
    index_of_interest, 
    crnn_model, 
    save_dir, 
    test_generator, 
    df_test,  
    test_offset=12
):
    print(f'Saving metric for contributor : {contributor}')
    times = {
        '5': 0,
        '10': 1,
        '15': 2,
        '20': 3,
        '25': 4,
        '30': 5,
        '35': 6,
        '40': 7,
        '45': 8,
        '50': 9,
        '55': 10,
        '60': 11
    }

    pred_df = run_predictions(crnn_model, times, test_generator, df_test)
    pred_df = pred_df.resample('1S').ffill()
    pred_df = pred_df.loc[index_of_interest][test_offset:].dropna()
    df_metric = compute_metric(times, pred_df)
    
    dump_df(df_metric, os.path.join(save_dir, f'{contributor}_metric.csv'))

    for time, time_idx in times.items():
        drop_time_columns = list(filter(lambda t: t != time_idx, times.values()))
        drop_columns = ['bolus', 'basal', 'carbs']
        drop_columns += list(map(lambda t: f'pred_cgm_{t}', drop_time_columns))
        pred_t_df = pred_df.drop(columns=drop_columns)
        pred_t_df.rename(columns={'cgm': 'true_BGL', f'pred_cgm_{time_idx}': 'predicted_BGL'}, inplace=True)
        dump_df(pred_t_df, os.path.join(save_dir, f'1_{contributor}_{time}.csv'))
        
    return df_metric

In [ ]:
def transfer_train_model(all_train_df, all_test_df, epochs=1000, shuffle=False):
    COLUMNS = ['carbs', 'bolus', 'basal', 'glucose_level']
    cb_lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor= 0.1, patience=15, min_lr= 1e-9)
    cb_early_stopping = EarlyStopping(
        monitor='val_loss',
        min_delta=0,
        patience=50,
        verbose=0,
        mode='auto',
        baseline=None,
        restore_best_weights=True
    )
    
    now = datetime.datetime.now()
    current_time = now.strftime('%Y_%m_%d_%H_%M')
    save_dir = os.path.join('output', current_time)
    os.makedirs(save_dir)
    metrics = []
    
    for train_contrib_id, test_contrib_id in zip(all_train_df.keys(), all_test_df.keys()):
        crnn_model = load_model('rcnn_pretrain.h5', custom_objects={'DeltaLSTM': DeltaLSTM})

        df_train = all_train_df[train_contrib_id]
        df_train['glucose_level'] = df_train['glucose_level'].interpolate('linear')
        df_train, train_interest_index = align_series(df_train)
        split_idx = int(len(df_train) * 0.8)
        df_val = df_train.iloc[split_idx+48:]
        df_train = df_train.iloc[:split_idx]
        X_train, Y_train = lag_target(df_train[COLUMNS])
        train_generator = make_generator(X_train.values, Y_train)
        
        
        df_test = all_test_df[test_contrib_id]
        df_test, test_interest_index = align_series(df_test)
        df_test = pd.concat([df_train.dropna().iloc[-12:], df_test], axis=0)
        
        X_val, Y_val = lag_target(df_val[COLUMNS])
        test_generator = make_generator(X_val.values, Y_val)
        
        print(f'Fitting contributor : {train_contrib_id}')
        
        crnn_model.fit_generator(train_generator,
                                 steps_per_epoch=len(train_generator),
                                 epochs=epochs,
                                 validation_data=test_generator,
                                 shuffle=shuffle,
                                 callbacks=[cb_lr_reducer, cb_early_stopping],
                                 verbose=1)
        
        evaluation_generator = SmoothedTimeseriesGenerator(
            data=df_test[COLUMNS].values, 
            targets=np.zeros((len(df_test), 12), dtype=np.float32), 
            length=24, 
            sampling_rate=1, 
            batch_size=32
        )
        
        df_metric = evaluate(test_contrib_id, test_interest_index, crnn_model, save_dir, evaluation_generator, df_test[COLUMNS])
        metrics.append(df_metric)
        
    print('Computing mean metric')
    df_metrics = pd.concat(metrics)
    df_metrics = df_metrics.groupby('Times').mean()
    dump_df(df_metrics, os.path.join(save_dir, 'all_contrib_mean_metric.csv'))

In [ ]:
transfer_train_model(all_df_train_stage2, all_df_test_stage2)